In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import squarify

In [ ]:
df = pd.read_csv("jumia_shoes_cleaned.csv")

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 2000)
pd.set_option('display.expand_frame_repr', True)

In [ ]:
print("=== Inspection part ===")
print(df.head())
print("------------------------------")
print(df.tail())
print("------------------------------")
df.info()
print("------------------------------")
print(df.isna().sum())
print("------------------------------")
print("Number of duplicates:", df.duplicated().sum())
print("==============================")

In [ ]:
print("=== Cleaning part ===")
#drop the duplicated rows
df = df.drop_duplicates()
print("Duplicates after drop:", df.duplicated().sum())
print("------------------------------")

In [ ]:
# Strip whitespace
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()

In [ ]:
# clean price and preprice from EGP
def parse_price_avg(val):
    if pd.isna(val):
        return np.nan
    s = str(val)
    s = re.sub(r'(?i)egp', '', s)
    s = s.replace(',', '')
    s = s.strip()
    if not s:
        return np.nan

    parts = [p.strip() for p in s.split('-')]
    nums = []
    for p in parts:
        if p == '':
            continue
        try:
            nums.append(float(p))
        except ValueError:
            continue

    if not nums:
        return np.nan
    return sum(nums) / len(nums) #multiple values -> average

In [ ]:
# Clean price and preprice
df['price'] = df['price'].apply(parse_price_avg)
df['preprice'] = df['preprice'].apply(parse_price_avg)

In [ ]:
# Maps specific textual values of main_material
material_map = {
    # LEATHER
    'Leather': 'leather',
    'جلد': 'leather',

    # TEXTILE
    'Textile': 'textile',
    'textiles': 'textile',
    'fabric': 'textile',
    'قماش': 'textile',
    'non-woven water-resistant fabric (polypropylene) with clear pvc': 'textile',

    # SYNTHETIC
    'Synthetic': 'synthetic',
    'synthetics': 'synthetic',
    'synthetic upper': 'synthetic',
    'ethylene': 'synthetic',
    'ethyle-vinyl-acetate': 'synthetic',
    'polyester': 'synthetic',
    'other material': 'synthetic',
    'plastic': 'synthetic',

    # MESH
    'Mesh': 'mesh',
    'mesh upper': 'mesh',
    'synthetic mesh upper': 'mesh',
    'mesh/pp/tpu': 'mesh',

    # FOAM / EVA
    'eva': 'foam',
    'eva foam': 'foam',
    'eva 100%': 'foam',
    'eva sockliner': 'foam',
    'Foam': 'foam',
    'foam & mesh': 'foam',

    # SUEDE
    'Suede': 'suede',
    'suede materials': 'suede',

    # CANVAS
    'Canvas': 'canvas',

    # SPECIAL MATERIALS
    'flyknit': 'knit',
    'Knit': 'knit',

    # RUBBER
    'Rubber': 'rubber',
    'perforated tongue rubber outsole for artificial turf': 'rubber',

    # MIXED / UNKNOWN
    'Mixed': 'mixed',
    'mix': 'mixed',
    'mixed upper material': 'mixed',
    'zsmc': 'mixed',

    'polyurethane 100%': 'polyurethane',
    'polyurethane': 'polyurethane',
    'pu': 'polyurethane',
    'polyvinyl chloride': 'polyurethane',
}

In [ ]:
df['main_material'] = df['main_material'].replace(material_map)
df['main_material'] = df['main_material'].fillna('mixed')

In [ ]:
# Normalize main_material:Strip spaces + lowercase
df['main_material'] = df['main_material'].str.lower().str.strip()

In [ ]:
# Remove values longer than 30 characters
df['main_material'] = df['main_material'].apply(
    lambda x: np.nan if isinstance(x, str) and len(x) > 30 else x
)

In [ ]:
#Replace very rare materials (frequency < 1%) with 'mixed'
freq = df['main_material'].value_counts(normalize=True)
rare_values = freq[freq < 0.01].index
df['main_material'] = df['main_material'].replace(rare_values, 'mixed')
df['main_material'] = df['main_material'].fillna('mixed')

In [ ]:
#Drop rows with empty/NaN sku
empty_mask = df['sku'].isna() | df['sku'].astype(str).str.match(r'^\s*$')
print("Empty SKU rows:", empty_mask.sum())
df = df[~empty_mask]
empty_mask_after = df['sku'].isna() | df['sku'].astype(str).str.match(r'^\s*$')
print("Empty SKU rows after drop:", empty_mask_after.sum())

In [ ]:
# Apply material mapping again
df['main_material'] = df['main_material'].replace(material_map)
df['main_material'] = df['main_material'].fillna('mixed')

In [ ]:
#Clean and fill brand with most common value
df['brand'] = df['brand'].astype(str).str.strip().str.lower()
df['brand'] = df['brand'].replace("", np.nan)
df['brand'] = df['brand'].replace('enjoy free delivery on orders above 400 egp.', np.nan)
most_common_brand = df['brand'].mode()[0]
df['brand'] = df['brand'].fillna(most_common_brand)

In [ ]:
# Remove '%' sign
df['discount'] = df['discount'].str.replace('%', '', regex=False)
df['discount'] = df['discount'].replace('', np.nan).astype(float)

In [ ]:
# Rows where discount is NaN: set preprice = price
mask_disc_nan = df['discount'].isna()
df.loc[mask_disc_nan, 'preprice'] = df.loc[mask_disc_nan, 'price']
df['discount'] = df['discount'].fillna(0.0)

In [ ]:
#Convert discount from percentage to fraction
df['discount'] = df['discount'] / 100.0

In [ ]:
# Ensure price and preprice are float
df['price'] = df['price'].astype(float)
df['preprice'] = df['preprice'].astype(float)

In [ ]:
print(df[['price', 'preprice', 'discount']].head())
print("------------------------------")

In [ ]:
# Drop unneeded columns: color, model, shop_type.
if 'color' in df.columns:
    df = df.drop(columns=['color'])

In [ ]:
if 'model' in df.columns:
    df = df.drop(columns=['model'])

In [ ]:
# Drop shop_type if exists
if 'shop_type' in df.columns:
    df = df.drop(columns=['shop_type'])

In [ ]:
# remove arabic in selected columns
arabic_pattern = re.compile(r'[\u0600-\u06FF]+')

In [ ]:
def remove_arabic(text):
    if isinstance(text, str):
        text = arabic_pattern.sub('', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

In [ ]:
for col in ['title', 'main_material', 'model']:
    if col in df.columns:
        df[col] = df[col].apply(remove_arabic)
        df[col] = df[col].replace('', np.nan)

In [ ]:
if 'main_material' in df.columns:
    df['main_material'] = df['main_material'].fillna('unknown')

In [ ]:
if 'model' in df.columns:
    df['model'] = df['model'].fillna('unknown')

In [ ]:
#-----------------------------------------------------------------
#Final inspection after cleaning
print("=== After cleaning (summary) ===")
print(df.head())
print("------------------------------")
df.info()
print("------------------------------")
print(df.isna().sum())
#-----------------------------------------------------------------
df.to_csv("jumia_shoes_cleaned.csv", index=False)
print("Cleaned data saved to 'jumia_shoes_cleaned.csv'")
#-----------------------------------------------------------------
print("=== visualization part ===")

In [ ]:
# Histogram for Price Distribution
# Shows How product prices are distributed (cheap vs expensive)
# Detect skewness & price concentration + possible outliers
plt.figure(figsize=(10,5))
sns.histplot(df["price"], kde=True)
plt.title("Price Distribution")
plt.xlabel("Price")
plt.ylabel("Count")
plt.show()

In [ ]:
# Boxplot for Price Outliers
# Shows Extreme high or low prices (outliers)
# Helps detect incorrect prices or unusual expensive items
plt.figure(figsize=(8,4))
sns.boxplot(df["price"])
plt.title("Price Boxplot")
plt.show()

In [ ]:
# Bar Chart for Top 10 Brands by Count
# Shows Most frequent brands in the dataset
# Understand market dominance & popular brands
top_brands = df["brand"].value_counts().head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_brands.index, y=top_brands.values)
plt.xticks(rotation=45)
plt.title("Top 10 Brands by Product Count")
plt.ylabel("Count")
plt.show()

In [ ]:
# Bar Chart for Average Price per Brand
# Shows Which brands have higher average price
# Understand pricing strategy differences between brands
avg_price = df.groupby("brand")["price"].mean().sort_values(ascending=False).head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=avg_price.index, y=avg_price.values)
plt.xticks(rotation=45)
plt.ylabel("Average Price")
plt.title("Top Brands by Average Price")
plt.show()

In [ ]:
# Bar Chart for Material Frequency
# Shows How many shoes are made from each material
# Understand product manufacturing trends
plt.figure(figsize=(10,5))
sns.countplot(x="main_material", data=df)
plt.title("Product Count by Main Material")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Heatmap for Correlation between price, preprice & discount
# Shows Correlation strength between numeric variables
# Understand if expensive shoes get bigger discounts
corr = df[["price", "preprice", "discount"]].corr()
plt.figure(figsize=(7,5))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# Bar Chart for Average Discount per Brand
# Shows Which brands offer highest average discount
# Understand sales strategy per brand
avg_disc = df.groupby("brand")["discount"].mean().sort_values(ascending=False).head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=avg_disc.index, y=avg_disc.values)
plt.title("Top 10 Brands with Highest Average Discount")
plt.xticks(rotation=45)
plt.ylabel("Average Discount")
plt.show()

In [ ]:
# Pie Chart for Distribution of gender
# Shows Percentage of shoes made from each gender
gender_counts = df["gender"].value_counts()
plt.figure(figsize=(8,8))
plt.pie(gender_counts.values, labels=gender_counts.index, autopct="%1.1f%%")
plt.title("gender Distribution")
plt.show()

In [ ]:
# Treemap for Count of Products by gender Category
# Shows How many products belong to gender
# Helps visualize category proportions
material_counts = df["main_material"].value_counts()
plt.figure(figsize=(30,15))
squarify.plot(
    sizes = material_counts.values,
    label = material_counts.index,
    alpha = 0.4
)
plt.title("Treemap of Product Count by material Category")
plt.axis('off')
plt.show()

In [ ]:
# Word Cloud for Popular Brands
# Shows Brands that appear the most (bigger word = more products)
# Helps visualize brand popularity quickly
brand_text = " ".join(df["brand"].astype(str))
wc = WordCloud(width=1200, height=600, background_color="white").generate(brand_text)
plt.figure(figsize=(15,7))
plt.imshow(wc)
plt.axis("off")
plt.title("Word Cloud of Brands")
plt.show()